# A Note on GRaph THeory Applications in Image Processing: Flood Fill, Image Segmentation, Pallette Generation
source: https://medium.com/geekculture/a-note-on-graph-theory-applications-in-image-processing-flood-fill-image-segmentation-palette-68a7957aed8c

## Flood Fill
1. flood fill is an algorithm that looks for nodes that are connected to the start node and finds a path by which to change the target color
2. can be modeled as a graph traversal, representing the given image as a matrix and considering every pixel as a vertex that connects to pixels around it (8 connectedness)

In [2]:
from collections import deque

def find_path_dfs_iter(self, start_id, target_id):
    """
    Use DFS with a stack to find a path from start_id to target_id.
    """

    if not self.contains_id(start_id) or not self.contains_id(target_id):
        raise KeyError("One or both vertices are not in the graph!")


    # stack of vertices to visit next
    stack = deque()
    stack.append(self.get_vertex(start_id))

    # vertex keys we've seen before and their paths from the start vertex
    path_to_target = {
        start_id: [start_id]
    }

    # while stack is not empty
    while stack:
        current_vertex_obj = stack.pop() # vertex obj to visit next
        current_vertex_id = current_vertex_obj.get_id()

        # found target, can stop the loop early
        if current_vertex_id == target_id:
            break

        neighbors = current_vertex_obj.get_neighbors()
        for neighbor in neighbors:
            if neighbor.get_id() not in path_to_target:
                stack.append(neighbor)
                # print(vertex_id_to_path)

                current_path = path_to_target[current_vertex_id]
                # extend the path by 1 vertex
                next_path = current_path + [neighbor.get_id()]
                path_to_target[neighbor.get_id()] = next_path

    if target_id not in path_to_target: # path not found
        return None

    return path_to_target[target_id]

## Image Segmentation
1. Felzenswalb segmentation method based on minimum spanning tree
2. edges are considered in increasing order of weight
    * endpoints merged into a region if this doesn't cause a cycle in the graph and also based on simialrity
    * pixel similarity judged via heuristic

source: https://sandipanweb.wordpress.com/2018/02/25/graph-based-image-segmentation-in-python/


In [3]:
def minimum_spanning_tree_kruskal(self):
  """
  Use Kruskal's Algorithm to return a list of edges, as tuples of 
  (start_id, dest_id, weight) in the graph's minimum spanning tree.
  """
  # Create a list of all edges in the graph, sort them by weight
  # from smallest to largest
  edges = []
  for vertex in self.get_vertices():
      # print(vertex.get_neighbors_with_weights())
      for neighbor, weight in vertex.get_neighbors_with_weights():
          edges.append((vertex.get_id(), neighbor.get_id(), weight))
  edges = sorted(edges, key=lambda x: x[2])

  # Create a dictionary `parent_map` to map vertex -> its "parent". 
  # Initialize it so that each vertex is its own parent.
  parent_map = {x[0]:x[0] for x in edges}

  # Create an empty list to hold the solution (i.e. all edges in the final spanning tree)
  spanning_tree = []

  # While the spanning tree holds < V-1 edges, 
  while len(spanning_tree) <= len(edges)-1:
      # get the smallest edge. 
      current = edges.pop(0)
      (vertex1, vertex2, weight) = current
      # If the two vertices connected by the edge are in different sets 
      # (i.e. calling `find()` gets two different roots), then it will not create a cycle, 
      if self.find(parent_map, vertex1) != self.find(parent_map, vertex2):
          # add it to the solution set and call `union()` on the two vertices.
          spanning_tree.append(current)
          self.union(parent_map, vertex1, vertex2)
      else:
          continue

  # Return the solution list.
  return spanning_tree

# Color Quantization
1. vertices contain information required by comparision heuristic, while edges indicate connected neighbors
2. union-find data structure